In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
root_path = '/content/drive/MyDrive/dataset/'

In [3]:
import os
import numpy as np
import pandas as pd

IMAGE_DIR = root_path + 'new_2019_images'
panda_path = root_path + 'new_2019.csv'
TEST_IMAGE_DIR = root_path + 'ISIC2018_Task3_Test_Input'
test_panda_path = root_path + 'val_class.csv'

In [4]:
print(len(os.listdir(IMAGE_DIR)))

11987


In [5]:
print(len(os.listdir(TEST_IMAGE_DIR)))

1514


In [6]:
print(f'This is the image dir: {IMAGE_DIR}')
print(f'This is the csv filepath: {panda_path}')

This is the image dir: /content/drive/MyDrive/dataset/new_2019_images
This is the csv filepath: /content/drive/MyDrive/dataset/new_2019.csv


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [8]:
def preprocess(df):
  for index, img in enumerate(df.image):
    img = img+'.jpg'
    df.image[index]=img
  # df.drop(['UNK'], axis=1, inplace=True)

In [9]:
def train_val_split(df, test_len=1000, val_ratio=0.2):
    val_rows = (np.random.rand(int(val_ratio*df.shape[0]))*df.shape[0]).astype(int)
    val_df = df.iloc[val_rows]
    df.drop(val_rows, axis=0, inplace=True)
    val_df = val_df.reset_index().drop(['index'], axis=1)
    df = df.reset_index().drop(['index'], axis=1)
    return df, val_df

In [10]:
train_df = pd.read_csv(panda_path)
preprocess(train_df)
val_df = pd.read_csv(test_panda_path)
preprocess(val_df)
labels=list(train_df.columns[1:])
print(labels)
train_df.head()

<ipython-input-8-95ecc90bf68d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.image[index]=img
<ipython-input-8-95ecc90bf68d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.image[index]=img


['MEL', 'NV', 'BCC', 'BKL', 'UNK']


,image,MEL,NV,BCC,BKL,UNK
0,ISIC_0068849.jpg,0.0,1.0,0.0,0.0,0.0
1,ISIC_0068458.jpg,0.0,1.0,0.0,0.0,0.0
2,ISIC_0000011.jpg,0.0,1.0,0.0,0.0,0.0
3,ISIC_0010585.jpg,0.0,1.0,0.0,0.0,0.0
4,ISIC_0025080.jpg,0.0,1.0,0.0,0.0,0.0


In [11]:
val_df.head()

,image,MEL,NV,BCC,BKL,UNK
0,ISIC_0034524.jpg,0.0,1.0,0.0,0.0,0.0
1,ISIC_0034525.jpg,0.0,1.0,0.0,0.0,0.0
2,ISIC_0034526.jpg,0.0,0.0,0.0,1.0,0.0
3,ISIC_0034527.jpg,0.0,1.0,0.0,0.0,0.0
4,ISIC_0034528.jpg,0.0,1.0,0.0,0.0,0.0


In [14]:
def get_train_gen(df, img_path=IMAGE_DIR, target_size=(224, 224)):
  data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                horizontal_flip=True,
                                width_shift_range=0.2,
                                height_shift_range=0.2)
  return data_gen.flow_from_dataframe(dataframe=df, directory=img_path,
                                      x_col='image', y_col=list(df.columns)[1:],
                                      batch_size=128, shuffle=True, class_mode='raw',
                                      target_size=target_size)

def get_val_gen(val_df, img_path=IMAGE_DIR, test_img_path=TEST_IMAGE_DIR, target_size=(224, 224)):
  data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
  val = data_gen.flow_from_dataframe(dataframe=val_df, directory=TEST_IMAGE_DIR,
                                      x_col='image', y_col=list(val_df.columns)[1:],
                                      batch_size=128, shuffle=True, class_mode='raw',
                                      target_size=target_size)
  return val

In [15]:
train_generator = get_train_gen(train_df)
valid_generator = get_val_gen(val_df)

Found 11987 validated image filenames.
Found 1512 validated image filenames.


In [16]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

def resnet_block(input, filters, kernel_size=(3, 3), strides=(1, 1)):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(input)
    shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def build_resnet18(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Residual blocks
    x = resnet_block(x, 64)
    x = resnet_block(x, 64)

    x = resnet_block(x, 128, strides=(2, 2))
    x = resnet_block(x, 128)

    x = resnet_block(x, 256, strides=(2, 2))
    x = resnet_block(x, 256)

    x = resnet_block(x, 512, strides=(2, 2))
    x = resnet_block(x, 512)

    x = GlobalAveragePooling2D()(x)
    outputs = Dense(5, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Example usage
input_shape = (224, 224, 3)
model = build_resnet18(input_shape)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 112, 112, 64)         0         ['batch_normalization[0][0

In [17]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
history = model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID, epochs=30)

Epoch 1/30
93/93 [==============================] - 4919s 53s/step - loss: 1.3905 - accuracy: 0.4094 - val_loss: 1.5101 - val_accuracy: 0.4077
Epoch 2/30
93/93 [==============================] - 339s 4s/step - loss: 1.2610 - accuracy: 0.4648 - val_loss: 1.0217 - val_accuracy: 0.6023
Epoch 3/30
93/93 [==============================] - 341s 4s/step - loss: 1.2044 - accuracy: 0.4896 - val_loss: 0.8999 - val_accuracy: 0.6385
Epoch 4/30
93/93 [==============================] - 363s 4s/step - loss: 1.1629 - accuracy: 0.5123 - val_loss: 1.0225 - val_accuracy: 0.5753
Epoch 5/30
93/93 [==============================] - 336s 4s/step - loss: 1.1310 - accuracy: 0.5263 - val_loss: 0.9415 - val_accuracy: 0.6364
Epoch 6/30
93/93 [==============================] - 329s 4s/step - loss: 1.1017 - accuracy: 0.5413 - val_loss: 1.2382 - val_accuracy: 0.5334
Epoch 7/30
93/93 [==============================] - 325s 3s/step - loss: 1.0772 - accuracy: 0.5527 - val_loss: 1.2422 - val_accuracy: 0.4950
Epoch 8/30


UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/dataset/new_2019_images/ISIC_0055036.jpg'
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/data_adapter.py", line 917, in wrapped_generator
    for da

In [ ]:
model.save('/content/drive/MyDrive/dataset/cnn_model_test.h5')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
#copied from Coursera util package
from keras.preprocessing import image
from sklearn.metrics import roc_auc_score, roc_curve
from tensorflow.compat.v1.logging import INFO, set_verbosity
import cv2

def get_roc_curve(labels, predicted_vals, generator):
    auc_roc_vals = []
    for i in range(len(labels)):
        try:
            gt = generator.labels[:, i]
            pred = predicted_vals[:, i]
            auc_roc = roc_auc_score(gt, pred)
            auc_roc_vals.append(auc_roc)
            fpr_rf, tpr_rf, _ = roc_curve(gt, pred)
            plt.figure(1, figsize=(10, 10))
            plt.plot([0, 1], [0, 1], 'k--')
            plt.plot(fpr_rf, tpr_rf,
                     label=labels[i] + " (" + str(round(auc_roc, 3)) + ")")
            plt.xlabel('False positive rate')
            plt.ylabel('True positive rate')
            plt.title('ROC curve')
            plt.legend(loc='best')
        except:
            print(
                f"Error in generating ROC curve for {labels[i]}. "
                f"Dataset lacks enough examples."
            )
    plt.show()
    return auc_roc_vals

In [ ]:
preds = model.predict_generator(valid_generator)

In [ ]:
import matplotlib.pyplot as plt
auc_rocs = get_roc_curve(labels, preds, valid_generator)

In [ ]:
y_true, y_pred = [], []
for pred, arr in zip(preds, valid_generator.labels):
    y_true.append(np.argmax(arr))
    y_pred.append(np.argmax(pred))
y_true = np.array(y_true)
y_pred = np.array(y_pred)
print("y_true:", y_true)
print("y_pred:", y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
class_labels = list(val_df.columns[1:])  # adjust the index if needed
report = classification_report(y_true, y_pred, target_names=class_labels)

In [ ]:
print(report)

In [ ]:
model2 = build_resnet18(input_shape)
model2.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
history = model2.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID, epochs=20)

In [ ]:
y_true, y_pred = [], []
for pred, arr in zip(preds, valid_generator.labels):
    y_true.append(np.argmax(arr))
    y_pred.append(np.argmax(pred))
y_true = np.array(y_true)
y_pred = np.array(y_pred)
print("y_true:", y_true)
print("y_pred:", y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
class_labels = list(val_df.columns[1:])  # adjust the index if needed
report = classification_report(y_true, y_pred, target_names=class_labels)

In [ ]:
print(report)